# Lab 12: Predictive model with Random Forest

As always, we create a SparkContext/HiveContext.

In [1]:
# Set up Spark Context
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import *

SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf()
conf.set('spark.executor.instances', 15)
conf.set('spark.sql.autoBroadcastJoinThreshold', 100*1024*1024)  # 100MB for broadcast join
sc = SparkContext('yarn-client', 'Spark-lab12', conf=conf)

from pyspark.sql import HiveContext
hc = HiveContext(sc)
hc.sql("use demo")

DataFrame[result: string]

In [2]:
def eval_metrics(lap):
    tp = float(len(lap[(lap['label']==1) & (lap['prediction']==1)]))
    tn = float(len(lap[(lap['label']==0) & (lap['prediction']==0)]))
    fp = float(len(lap[(lap['label']==0) & (lap['prediction']==1)]))
    fn = float(len(lap[(lap['label']==1) & (lap['prediction']==0)]))
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    return {'precision': precision, 'recall': recall, 'accuracy': accuracy}

As with lab11:
1. Load the feature matrix created in lab 10 into a Spark dataframe called 'fm'
2. Split into two dataframes - train (2011-2013) and test (only 2014)

In [3]:
fm = hc.read.format("orc").load("/tmp/fm")
trainData = fm.filter(fm.year<=2013).cache()
testData = fm.filter(fm.year>=2014).cache()
print trainData.count(), testData.count()

426306 150155


First, create a pipline very similar to lab 11, only this time we use Random Forest instead of Logistic regression.
For parameters to the random forest, you can use: numTrees=100, maxDepth=4, maxBins=50

In [4]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

inx1 = StringIndexer(inputCol="category", outputCol="cat-inx")
inx2 = StringIndexer(inputCol="dayofweek", outputCol="dow-inx")
inx3 = StringIndexer(inputCol="district", outputCol="dis-inx")
inx4 = StringIndexer(inputCol="neighborhood", outputCol="ngh-inx")
inx5 = StringIndexer(inputCol="resolved", outputCol="label")
vecAssembler = VectorAssembler(inputCols =["month", "hour", "prcp", "tmin", "tmax", \
                                           "cat-inx", "dow-inx", "dis-inx", "ngh-inx"], 
                               outputCol="features")
rf = RandomForestClassifier(numTrees=100, maxDepth=4, maxBins=50, labelCol="label", seed=42)

# Create pipeline
pipeline_rf = Pipeline(stages=[inx1, inx2, inx3, inx4, inx5, vecAssembler, rf])

# Train model
model_rf = pipeline_rf.fit(trainData)

# Evaluate results on testData
results = model_rf.transform(testData)
lap = results.select("label", "prediction").toPandas()
print eval_metrics(lap)


{'recall': 0.6399471482052411, 'precision': 0.7563604815095977, 'accuracy': 0.7945256568212846}


Now modify this pipeline to also add up to 50 features corresponding to words in the "description" field. Use Tokenizer and HashingTF:

In [5]:
from pyspark.ml.feature import Tokenizer, HashingTF

# Create modified pipeline
inx1 = StringIndexer(inputCol="category", outputCol="cat-inx")
inx2 = StringIndexer(inputCol="dayofweek", outputCol="dow-inx")
inx3 = StringIndexer(inputCol="district", outputCol="dis-inx")
inx4 = StringIndexer(inputCol="neighborhood", outputCol="ngh-inx")
inx5 = StringIndexer(inputCol="resolved", outputCol="label")
parser = Tokenizer(inputCol="description", outputCol="words")
hashingTF = HashingTF(numFeatures=50, inputCol="words", outputCol="hash-inx")
vecAssembler = VectorAssembler(inputCols =["month", "hour", "prcp", "tmin", "tmax", \
                                           "cat-inx", "dow-inx", "dis-inx", "ngh-inx", "hash-inx"], 
                               outputCol="features")
rf = RandomForestClassifier(numTrees=100, maxDepth=4, maxBins=50, labelCol="label", seed=42)
pipeline_rf = Pipeline(stages=[inx1, inx2, inx3, inx4, inx5, parser, hashingTF, vecAssembler, rf])

Train the random forest and evaluate the results using the eval_metrics() function:

In [6]:
# Train model
model_rf = pipeline_rf.fit(trainData)

# Evaluate results on testData
results = model_rf.transform(testData)
lap = results.select("label", "prediction").toPandas()
print eval_metrics(lap)

{'recall': 0.7081222931806503, 'precision': 0.7964293085655315, 'accuracy': 0.8283906629815857}


In addition to the overall accuracy metrics, we now want to plot the accuacy of prediction per each neighborhood. In order to do this, we first use ESRI's HIVE UDFs: ST_X, ST_Y nad ST_Centroid to compute the longitude/latitude centroid of each neighborhood in San Francisco:

In [7]:
hc.sql("add jar /home/jupyter/notebooks/jars/guava-11.0.2.jar")
hc.sql("add jar /home/jupyter/notebooks/jars/esri-geometry-api.jar")
hc.sql("add jar /home/jupyter/notebooks/jars/spatial-sdk-hive.jar")
hc.sql("add jar /home/jupyter/notebooks/jars/spatial-sdk-json.jar")

hc.sql("create temporary function ST_Centroid as 'com.esri.hadoop.hive.ST_Centroid'")
hc.sql("create temporary function ST_X as 'com.esri.hadoop.hive.ST_X'")
hc.sql("create temporary function ST_Y as 'com.esri.hadoop.hive.ST_Y'")

df_centroid = hc.sql("""
SELECT neighborho as neighborhood, 
       ST_X(ST_Centroid(sf_neighborhoods.shape)) as cent_longitude,
       ST_Y(ST_Centroid(sf_neighborhoods.shape)) as cent_latitude
FROM sf_neighborhoods
""")
df_centroid.cache()

DataFrame[neighborhood: string, cent_longitude: double, cent_latitude: double]

Now that we have the centroid for each neighborhood, we compute the accuracy of prediction specific to crimes within that neighborhood. 

Complete the code below to plot a map of san francisco, with markers in the centroid of each neighborhood showing that accuracy number.

In [8]:
import folium

from IPython.display import HTML
map_width=1000
map_height=600
sf_lat = 37.77
sf_long = -122.4

def inline_map(m, width=map_width, height=map_height):
    m.create_map()
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: {}px; height: {}px; '
                 'border: none"></iframe>'.format(srcdoc, width, height))
    return embed


n_list = results.select("neighborhood").distinct().toPandas()['neighborhood'].tolist()

df = results.select("neighborhood", "label", "prediction").toPandas()
map_sf = folium.Map(location=[sf_lat, sf_long], zoom_start=12, width=map_width, height=map_height)
for n in df_centroid.collect():
    if n.neighborhood in n_list:
        m = eval_metrics(df[df['neighborhood']==n.neighborhood])
        map_sf.simple_marker([n.cent_latitude, n.cent_longitude], \
                             popup = n.neighborhood + ": accuracy = %.2f" % m['accuracy'])

inline_map(map_sf) 